In [16]:
import joblib
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import lime
import lime.lime_tabular
from lime import lime_text
from sklearn.pipeline import make_pipeline

# Load model and data

## Load model and data

In [22]:
clf = joblib.load("./dockerfile/apps/model_new.pkl")
df = pd.read_csv("./data/ProductionData.csv")
df['qual_student'] = np.where(df['G3']>=15, 1, 0)
X = df.drop(['qual_student','G3'], axis=1)
y = df['qual_student']

X['schoolsup'] = X['schoolsup'].replace(to_replace=['no','yes'],value=[0,1])
X['famsup'] = X['famsup'].replace(to_replace=['no','yes'],value=[0,1])
X['activities'] = X['activities'].replace(to_replace=['no','yes'],value=[0,1])
X['higher'] = X['higher'].replace(to_replace=['no','yes'],value=[0,1])
X['internet'] = X['internet'].replace(to_replace=['no','yes'],value=[0,1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

include = ['studytime','failures','schoolsup','famsup','activities','freetime',
           'higher','Dalc','Walc','internet','health','absences']
exclude = []
X_train = X_train.drop(columns=X_train.columns.difference(include))
X_train = X_train.drop(columns=exclude)
X_test = X_test.drop(columns=X_test.columns.difference(include))
X_test = X_test.drop(columns=exclude)

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

## Analyze predicted results

In [36]:
predict_fn_rf = lambda x: clf.predict_proba(x).astype(float)
X = X_train.values
explainer = lime.lime_tabular.LimeTabularExplainer(X,feature_names=X_train.columns,class_names=['Qualified','Not Qualified'],kernel_width=5)

In [41]:
choosen_instance = X_test.iloc[0]
clf.predict(X_test.iloc[0].reshape(1, -1))
exp = explainer.explain_instance(choosen_instance, predict_fn_rf,num_features=12)
exp.show_in_notebook(show_all=False)

AttributeError: 'Series' object has no attribute 'reshape'